In [2]:
# 把单条 XY 表转成 OJP 外轮廓的 CSV
import os
import pandas as pd
import numpy as np
import pygmt

# -------- GMT 全局配置（可按需增减）--------
pygmt.config(
    COLOR_FOREGROUND="gray",  # CPT 前景色（> 最高 z）
    # COLOR_BACKGROUND="navy",
    # COLOR_NAN="white",
)

# ===============【需要时在这里改 · 全局参数】===============
REGION = [150, 174, -10, 6]
PROJ   = "M16c"

SITES_CSV = r"E:\wjy\Gravity\data\Drilled_Holes.csv"

USE_GEBCO   = False
GEBCO_PATH  = r"E:\wjy\Gravity\data\GEBCO_2021.nc"

SEA_ONLY    = True

# --- 颜色带（SCM/vik）---
CPT_NAME    = "vik"
CPT_MODE    = "half_neg"      # "half_neg" 只海盆；"symmetric" 海陆对称
CPT_STEP    = 100
# 将色标范围压缩为 -6000~0 m，使 vik 中性色靠近 -3000 m，更突出 OJP
CPT_SERIES_MANUAL = [-7000, 0, 100]
CPT_FILE    = "vik_used.cpt"  # 固定输出 CPT 文件名

LABEL_OFFSET = "0c/0.20c"
LABEL_FONT   = "9p,Helvetica-Bold,black"   # 大小, 字体, 颜色
LABEL_FILL   = "white@60"
LABEL_BOX    = "2p/2p"
LABEL_PEN    = "0.2p,black"

HIGHLIGHT_DSDP = {288, 289, 586}
HIGHLIGHT_ODP  = {803, 804, 805, 806, 807, 1183, 1184, 1185, 1186, 1187}

DRAW_LABELS    = False   # 只画符号不过站位号

# OJP 轮廓线：用 WebPlotDigitizer 数出的 CSV
LINES_CSV   = r"E:\wjy\Gravity\out\pic\OJP_outline_ToGMT.csv"
LINES_SHP   = ""

OUTBASE = r"E:\wjy\Gravity\out\pic\backup\OJP_compare"
# =========================================================

# 读取站位
sites = pd.read_csv(SITES_CSV)
colmap = {c.lower(): c for c in sites.columns}
for need in ["leg", "site", "latitude", "longitude"]:
    assert need in colmap, f"CSV 缺少字段: {need}"
sites = sites.rename(columns={
    colmap["leg"]: "leg",
    colmap["site"]: "site",
    colmap["latitude"]: "lat",
    colmap["longitude"]: "lon",
})
sites = sites.dropna(subset=["lat", "lon"])
sites = sites[
    (sites["lon"] >= REGION[0]) & (sites["lon"] <= REGION[1])
    & (sites["lat"] >= REGION[2]) & (sites["lat"] <= REGION[3])
]

def _program_from_leg(x):
    try:
        v = float(x)
    except Exception:
        return "Other"
    if 1 <= v <= 96:
        return "DSDP"
    if 100 <= v <= 210:
        return "ODP"
    return "Other"

sites["program"] = sites["leg"].apply(_program_from_leg)

def _site_num(s):
    try:
        return int(float(str(s).split()[0]))
    except Exception:
        return np.nan

sites["site_num"] = sites["site"].apply(_site_num)

# 底图格网
if USE_GEBCO and os.path.exists(GEBCO_PATH):
    grid = pygmt.grdcut(grid=GEBCO_PATH, region=REGION)
else:
    grid = pygmt.datasets.load_earth_relief(resolution="01m", region=REGION)

shade = pygmt.grdgradient(grid=grid, radiance=[270, 30])

grid_to_plot = grid
if SEA_ONLY:
    # 把 0 m 以上的值裁剪为 NaN，陆地用 coast 再统一着色
    grid_to_plot = pygmt.grdclip(grid=grid, above=[0, np.nan])

# --------- 生成 vik CPT 文件（锁定、带灰色前景）---------
with pygmt.config(COLOR_FOREGROUND="gray"):
    if CPT_SERIES_MANUAL is not None:
        pygmt.makecpt(
            cmap=CPT_NAME,
            series=CPT_SERIES_MANUAL,
            continuous=True,
            output=CPT_FILE,
            overrule_bg=True,
        )
    else:
        gmin = float(grid.min().values)
        gmax = float(grid.max().values)
        if CPT_MODE == "symmetric":
            M = max(abs(gmin), abs(gmax), 2000)
            pygmt.makecpt(
                cmap=CPT_NAME,
                series=[-M, M, CPT_STEP],
                continuous=True,
                output=CPT_FILE,
                overrule_bg=True,
            )
        else:  # half_neg
            lo = min(-8000, gmin)
            pygmt.makecpt(
                cmap=CPT_NAME,
                series=[lo, 0, CPT_STEP],
                continuous=True,
                output=CPT_FILE,
                overrule_bg=True,
            )
# ---------------------------------------------------------

fig = pygmt.Figure()
pygmt.config(
    MAP_FRAME_TYPE="plain",
    FORMAT_GEO_MAP="dddF",
    MAP_TICK_LENGTH_PRIMARY="0.15c",
    COLOR_NAN="white",
)

# 海底地形底图
fig.grdimage(
    grid=grid_to_plot,
    cmap=CPT_FILE,
    projection=PROJ,
    region=REGION,
    shading=shade,
)

# 海岸线 / 陆地
if SEA_ONLY:
    fig.coast(
        land="gray90",
        shorelines="0.25p,black",
        borders="1/0.15p,gray40",
        frame=["WSen", "xafg", "yafg"],
    )
else:
    fig.coast(
        shorelines="0.25p,black",
        borders="1/0.15p,gray40",
        frame=["WSen", "xafg", "yafg"],
    )

# OJP 轮廓线
if LINES_CSV and os.path.exists(LINES_CSV):
    dfL = pd.read_csv(LINES_CSV)
    required_cols = {"lon", "lat"}
    assert required_cols.issubset(dfL.columns), \
        f"LINES_CSV 需要包含列: {required_cols}"
    if "line_id" in dfL.columns:
        for lid, g in dfL.groupby("line_id"):
            fig.plot(x=g["lon"], y=g["lat"], pen="1.5p,black")
    else:
        fig.plot(x=dfL["lon"], y=dfL["lat"], pen="1.5p,black")

if LINES_SHP and os.path.exists(LINES_SHP):
    fig.plot(data=LINES_SHP, pen="1.5p,black")

# 站位符号
dsdp = sites[sites["program"] == "DSDP"]
odp  = sites[sites["program"] == "ODP"]
if not dsdp.empty:
    fig.plot(
        x=dsdp["lon"],
        y=dsdp["lat"],
        style="s0.25c",
        fill="white",
        pen="0.6p,black",
    )
if not odp.empty:
    fig.plot(
        x=odp["lon"],
        y=odp["lat"],
        style="s0.28c",
        fill="gold",
        pen="0.6p,black",
    )

# 站位编号（可选）
if DRAW_LABELS:
    def put_label(x, y, text):
        fig.text(
            x=float(x),
            y=float(y),
            text=str(text),
            justify="BC",
            offset=LABEL_OFFSET,
            font=LABEL_FONT,
            fill=LABEL_FILL,
            clearance=LABEL_BOX,
            pen=LABEL_PEN,
        )

    lab = sites[sites["site_num"].isin(HIGHLIGHT_DSDP.union(HIGHLIGHT_ODP))].copy()
    for _, r in lab.iterrows():
        put_label(r["lon"], r["lat"], int(r["site_num"]))

# 色标
fig.colorbar(
    cmap=CPT_FILE,
    position="JBC+w14c/0.45c+o0c/0.8c+h",
    frame="a1000f500+lBathymetry/topography (m)",
)

# 输出：PNG + EPS
fig.savefig(OUTBASE + ".png", dpi=300)
fig.savefig(OUTBASE + ".pdf")
print(f"Saved: {OUTBASE}.png / .eps")

sites.sort_values(["program", "site_num"]).to_csv(
    r"E:\wjy\Gravity\out\pic\OJP_sites_filtered.csv",
    index=False,
)
print("Saved: OJP_sites_filtered.csv")


Saved: E:\wjy\Gravity\out\pic\backup\OJP_compare.png / .eps
Saved: OJP_sites_filtered.csv


In [ ]:
#统一excel格式
import pandas as pd

# 1) 改成你自己的路径
raw_csv = r"E:\wjy\Gravity\out\pic\OJP_LINE.csv"

df = pd.read_csv(raw_csv, header=None, names=["lon", "lat"])

# 看一下前几行确认一下
print(df.head())

df["line_id"] = 1

# 4) 保存成最终给 GMT 用的文件
out_csv = r"E:\wjy\Gravity\out\pic\OJP_outline_ToGMT.csv"
df.to_csv(out_csv, index=False)

print("Saved:", out_csv)


In [ ]:
#合并测线表，统一格式
import pandas as pd
from pathlib import Path

base = Path(r"E:\wjy\Gravity\out\pic\data")

# 这里根据你文件名改路径；键是你希望在图上显示的 line_name
files = {
    # Inoue 2008 三个航次
    "EW95-11":          base / "OJP_Inueo_2008_EW95-11.csv",
    "Kaiei_KR05-01":    base / "OJP_Inueo_2008_kairei_KR05-01.csv",
    "KH98-01_a":        base / "OJP_Inueo_2008_MARU_KH98_01_a.csv",
    "KH98-01_b":        base / "OJP_Inueo_2008_MARU_KH98_01_b.csv",
    "KH98-01_c":        base / "OJP_Inueo_2008_MARU_KH98_01_c.csv",
    "KH98-01_d":        base / "OJP_Inueo_2008_MARU_KH98_01_d.csv",
    # Miura 2004 OBS 剖面
    "Miura_2004":       base / "OJP_Miura_2004.csv",

    # Mosher 1993 T. Washington / Joides Resolution（名字按你的实际文件名改）
    "Mosher_TW_A":      base / "OJP_Mosher_1993_T. Washington seismic_A.csv",
    "Mosher_TW_B":      base / "OJP_Mosher_1993_T.Washington_B.csv",
    "Mosher_Joides":    base / "OJP_Mosher_1993_Joides Resolution.csv",

    # 其它你描的 transect，比如 Nauru Basin、Lyra Basin
    "Lyra_Basin":       base / "OJP_Lyra Basin.csv",
    "Nauru_Basin":      base / "OJP_Nauru_Basin.csv",
}
def load_xy_csv(path: str) -> pd.DataFrame:
    """
    读取 WebPlotDigitizer 导出的 X,Y CSV：
    - 默认取前两列
    - 自动丢掉非数值行（比如可能的表头）
    返回一个 DataFrame，列为 lon, lat
    """
    raw = pd.read_csv(path, header=None)

    lon_raw = pd.to_numeric(raw.iloc[:, 0], errors="coerce")
    lat_raw = pd.to_numeric(raw.iloc[:, 1], errors="coerce")
    mask = lon_raw.notna() & lat_raw.notna()

    df = pd.DataFrame({
        "lon": lon_raw[mask].values,
        "lat": lat_raw[mask].values,
    })
    return df

all_list = []

for name, path in files.items():
    if not os.path.exists(path):
        raise FileNotFoundError(f"找不到文件: {path}")
    df_line = load_xy_csv(path)
    df_line["line_name"] = name
    all_list.append(df_line)
    print(f"{name}: {len(df_line)} points")

all_lines = pd.concat(all_list, ignore_index=True)
out_csv = os.path.join(base, "OJP_all_lines_ToGMT.csv")
all_lines.to_csv(out_csv, index=False)

print("\n合并完成，输出文件:", out_csv)
print(all_lines.head())

In [7]:
# ojp_fig1_style_final_viklocked.py
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import pygmt

# -------- GMT 全局配置 --------
pygmt.config(
    COLOR_FOREGROUND="gray",  # CPT 前景色（> 最高 z）
)

# ===============【全局参数】===============
REGION = [150, 174, -10, 6]
PROJ   = "M16c"

SITES_CSV = r"E:\wjy\Gravity\data\Drilled_Holes.csv"

USE_GEBCO   = False
GEBCO_PATH  = r"E:\wjy\Gravity\data\GEBCO_2021.nc"

SEA_ONLY    = True

# --- 颜色带（SCM/vik）---
CPT_NAME    = "vik"
CPT_MODE    = "half_neg"
CPT_STEP    = 100
CPT_SERIES_MANUAL = [-7000, 0, 100]
CPT_FILE    = "vik_used.cpt"

LABEL_OFFSET = "0c/0.20c"
LABEL_FONT   = "9p,Helvetica-Bold,black"
LABEL_FILL   = "white@60"
LABEL_BOX    = "2p/2p"
LABEL_PEN    = "0.2p,black"

HIGHLIGHT_DSDP = {288, 289, 586}
HIGHLIGHT_ODP  = {803, 804, 805, 806, 807, 1183, 1184, 1185, 1186, 1187}

DRAW_LABELS    = False

# OJP 外轮廓
LINES_CSV   = r"E:\wjy\Gravity\out\pic\OJP_outline_ToGMT.csv"
LINES_SHP   = ""

# 合并后的前人测线
MCS_LINES_CSV = r"E:\wjy\Gravity\out\pic\data\OJP_all_lines_ToGMT.csv"

OUTBASE = r"E:\wjy\Gravity\out\pic\backup\OJP_seismic_plot"
# =========================================================

# 读取站位
sites = pd.read_csv(SITES_CSV)
colmap = {c.lower(): c for c in sites.columns}
for need in ["leg", "site", "latitude", "longitude"]:
    assert need in colmap, f"CSV 缺少字段: {need}"
sites = sites.rename(columns={
    colmap["leg"]: "leg",
    colmap["site"]: "site",
    colmap["latitude"]: "lat",
    colmap["longitude"]: "lon",
})
sites = sites.dropna(subset=["lat", "lon"])
sites = sites[
    (sites["lon"] >= REGION[0]) & (sites["lon"] <= REGION[1])
    & (sites["lat"] >= REGION[2]) & (sites["lat"] <= REGION[3])
]

def _program_from_leg(x):
    try:
        v = float(x)
    except Exception:
        return "Other"
    if 1 <= v <= 96:
        return "DSDP"
    if 100 <= v <= 210:
        return "ODP"
    return "Other"

sites["program"] = sites["leg"].apply(_program_from_leg)

def _site_num(s):
    try:
        return int(float(str(s).split()[0]))
    except Exception:
        return np.nan

sites["site_num"] = sites["site"].apply(_site_num)

# 底图格网
if USE_GEBCO and os.path.exists(GEBCO_PATH):
    grid = pygmt.grdcut(grid=GEBCO_PATH, region=REGION)
else:
    grid = pygmt.datasets.load_earth_relief(resolution="01m", region=REGION)

shade = pygmt.grdgradient(grid=grid, radiance=[270, 30])

grid_to_plot = grid
if SEA_ONLY:
    grid_to_plot = pygmt.grdclip(grid=grid, above=[0, np.nan])

# --------- 生成 vik CPT 文件 ---------
with pygmt.config(COLOR_FOREGROUND="gray"):
    if CPT_SERIES_MANUAL is not None:
        pygmt.makecpt(
            cmap=CPT_NAME,
            series=CPT_SERIES_MANUAL,
            continuous=True,
            output=CPT_FILE,
            overrule_bg=True,
        )
    else:
        gmin = float(grid.min().values)
        gmax = float(grid.max().values)
        if CPT_MODE == "symmetric":
            M = max(abs(gmin), abs(gmax), 2000)
            pygmt.makecpt(
                cmap=CPT_NAME,
                series=[-M, M, CPT_STEP],
                continuous=True,
                output=CPT_FILE,
                overrule_bg=True,
            )
        else:
            lo = min(-8000, gmin)
            pygmt.makecpt(
                cmap=CPT_NAME,
                series=[lo, 0, CPT_STEP],
                continuous=True,
                output=CPT_FILE,
                overrule_bg=True,
            )
# ---------------------------------------------------------

fig = pygmt.Figure()
pygmt.config(
    MAP_FRAME_TYPE="plain",
    FORMAT_GEO_MAP="dddF",
    MAP_TICK_LENGTH_PRIMARY="0.15c",
    COLOR_NAN="white",
)

# 海底地形底图
fig.grdimage(
    grid=grid_to_plot,
    cmap=CPT_FILE,
    projection=PROJ,
    region=REGION,
    shading=shade,
)

# 海岸线 / 陆地
if SEA_ONLY:
    fig.coast(
        land="gray90",
        shorelines="0.25p,black",
        borders="1/0.15p,gray40",
        frame=["WSen", "xafg", "yafg"],
    )
else:
    fig.coast(
        shorelines="0.25p,black",
        borders="1/0.15p,gray40",
        frame=["WSen", "xafg", "yafg"],
    )

# OJP 外轮廓
if LINES_CSV and os.path.exists(LINES_CSV):
    dfL = pd.read_csv(LINES_CSV)
    required_cols = {"lon", "lat"}
    assert required_cols.issubset(dfL.columns), \
        f"LINES_CSV 需要包含列: {required_cols}"
    if "line_id" in dfL.columns:
        for lid, g in dfL.groupby("line_id"):
            fig.plot(x=g["lon"], y=g["lat"], pen="1.5p,black")
    else:
        fig.plot(x=dfL["lon"], y=dfL["lat"], pen="1.5p,black")

if LINES_SHP and os.path.exists(LINES_SHP):
    fig.plot(data=LINES_SHP, pen="1.5p,black")

# ======= 叠加前人地震测线 =======
if MCS_LINES_CSV and os.path.exists(MCS_LINES_CSV):
    mcs = pd.read_csv(MCS_LINES_CSV)
    required_cols = {"lon", "lat", "line_name"}
    assert required_cols.issubset(mcs.columns), \
        f"MCS_LINES_CSV 需要包含列: {required_cols}, 实际为 {mcs.columns}"

    style_map = {
        # 现代 MCS 主测线
        "Kaiei_KR05-01": "2p,black",
        "KH98-01_a":     "2p,white",
        "KH98-01_b":     "2p,white",
        "KH98-01_c":     "2p,white",
        "KH98-01_d":     "2p,white",
        "EW95-11":       "2p,purple",

        # OBS 宽角剖面
        "Miura_2004":    "2p,red",

        # 早期反射/折射：
        "Mosher_TW_A":   "2p,green,.-",
        "Mosher_TW_B":   "2p,green,.-",
        "Mosher_Joides": "1.5p,green",

        # 盆地边界
        "Lyra_Basin":    "2p,blue",
        "Nauru_Basin":   "2p,blue",
    }

    for name, g in mcs.groupby("line_name"):
        pen = style_map.get(name, "0.8p,gray50")
        fig.plot(
            x=g["lon"],
            y=g["lat"],
            pen=pen,
        )
# =====================================================

# 站位符号
dsdp = sites[sites["program"] == "DSDP"]
odp  = sites[sites["program"] == "ODP"]
if not dsdp.empty:
    fig.plot(
        x=dsdp["lon"],
        y=dsdp["lat"],
        style="s0.25c",
        fill="white",
        pen="0.6p,black",
    )
if not odp.empty:
    fig.plot(
        x=odp["lon"],
        y=odp["lat"],
        style="s0.28c",
        fill="gold",
        pen="0.6p,black",
    )

# 站位编号（可选）
if DRAW_LABELS:
    def put_label(x, y, text):
        fig.text(
            x=float(x),
            y=float(y),
            text=str(text),
            justify="BC",
            offset=LABEL_OFFSET,
            font=LABEL_FONT,
            fill=LABEL_FILL,
            clearance=LABEL_BOX,
            pen=LABEL_PEN,
        )

    lab = sites[sites["site_num"].isin(HIGHLIGHT_DSDP.union(HIGHLIGHT_ODP))].copy()
    for _, r in lab.iterrows():
        put_label(r["lon"], r["lat"], int(r["site_num"]))

# 色标
fig.colorbar(
    cmap=CPT_FILE,
    position="JBC+w14c/0.45c+o0c/0.8c+h",
    frame="a1000f500+lBathymetry/topography (m)",
)

# 输出：PNG + PDF
fig.savefig(OUTBASE + ".png", dpi=300)
fig.savefig(OUTBASE + ".pdf")
print(f"Saved: {OUTBASE}.png / .pdf")

sites.sort_values(["program", "site_num"]).to_csv(
    r"E:\wjy\Gravity\out\pic\OJP_sites_filtered.csv",
    index=False,
)
print("Saved: OJP_sites_filtered.csv")


Saved: E:\wjy\Gravity\out\pic\backup\OJP_seismic_plot.png / .pdf
Saved: OJP_sites_filtered.csv


In [10]:

import os
import pandas as pd
import numpy as np
import pygmt


pygmt.config(
    COLOR_FOREGROUND="gray", 
)


REGION = [150, 174, -10, 6]
PROJ   = "M16c"

SITES_CSV = r"E:\wjy\Gravity\data\Drilled_Holes.csv"

USE_GEBCO   = False
GEBCO_PATH  = r"E:\wjy\Gravity\data\GEBCO_2021.nc"

SEA_ONLY    = True


CPT_NAME    = "vik"
CPT_MODE    = "half_neg"
CPT_STEP    = 100

CPT_SERIES_MANUAL = [-7000, 0, 100]
CPT_FILE    = "vik_used.cpt"

LABEL_OFFSET = "0c/0.20c"
LABEL_FONT   = "9p,Helvetica-Bold,black"
LABEL_FILL   = "white@60"
LABEL_BOX    = "2p/2p"
LABEL_PEN    = "0.2p,black"

HIGHLIGHT_DSDP = {288, 289, 586}
HIGHLIGHT_ODP  = {803, 804, 805, 806, 807, 1183, 1184, 1185, 1186, 1187}

DRAW_LABELS    = False


LINES_CSV   = r"E:\wjy\Gravity\out\pic\data\OJP_outline_ToGMT.csv"
LINES_SHP   = ""


MCS_LINES_CSV = r"E:\wjy\Gravity\out\pic\data\OJP_all_lines_ToGMT.csv"

OUTBASE = r"E:\wjy\Gravity\out\pic\backup\OJP_seismic_plot"

sites = pd.read_csv(SITES_CSV)
colmap = {c.lower(): c for c in sites.columns}
for need in ["leg", "site", "latitude", "longitude"]:
    assert need in colmap, f"CSV 缺少字段: {need}"
sites = sites.rename(columns={
    colmap["leg"]: "leg",
    colmap["site"]: "site",
    colmap["latitude"]: "lat",
    colmap["longitude"]: "lon",
})
sites = sites.dropna(subset=["lat", "lon"])
sites = sites[
    (sites["lon"] >= REGION[0]) & (sites["lon"] <= REGION[1])
    & (sites["lat"] >= REGION[2]) & (sites["lat"] <= REGION[3])
]

def _program_from_leg(x):
    try:
        v = float(x)
    except Exception:
        return "Other"
    if 1 <= v <= 96:
        return "DSDP"
    if 100 <= v <= 210:
        return "ODP"
    return "Other"

sites["program"] = sites["leg"].apply(_program_from_leg)

def _site_num(s):
    try:
        return int(float(str(s).split()[0]))
    except Exception:
        return np.nan

sites["site_num"] = sites["site"].apply(_site_num)


if USE_GEBCO and os.path.exists(GEBCO_PATH):
    grid = pygmt.grdcut(grid=GEBCO_PATH, region=REGION)
else:
    # 用远程 earth_relief 数据
    grid = pygmt.datasets.load_earth_relief(resolution="01m", region=REGION)


shade = pygmt.grdgradient(grid=grid, radiance=[270, 30])

# 只保留海底，陆地用 coast 统一着色
grid_to_plot = grid
if SEA_ONLY:
    grid_to_plot = pygmt.grdclip(grid=grid, above=[0, np.nan])
# ===========================================================

# --------- 生成 vik CPT 文件 ---------
with pygmt.config(COLOR_FOREGROUND="gray"):
    if CPT_SERIES_MANUAL is not None:
        pygmt.makecpt(
            cmap=CPT_NAME,
            series=CPT_SERIES_MANUAL,
            continuous=True,
            output=CPT_FILE,
            overrule_bg=True,
        )
    else:
        gmin = float(grid.min().values)
        gmax = float(grid.max().values)
        if CPT_MODE == "symmetric":
            M = max(abs(gmin), abs(gmax), 2000)
            pygmt.makecpt(
                cmap=CPT_NAME,
                series=[-M, M, CPT_STEP],
                continuous=True,
                output=CPT_FILE,
                overrule_bg=True,
            )
        else:
            lo = min(-8000, gmin)
            pygmt.makecpt(
                cmap=CPT_NAME,
                series=[lo, 0, CPT_STEP],
                continuous=True,
                output=CPT_FILE,
                overrule_bg=True,
            )
# ---------------------------------------------------------

fig = pygmt.Figure()
pygmt.config(
    MAP_FRAME_TYPE="plain",
    FORMAT_GEO_MAP="dddF",
    MAP_TICK_LENGTH_PRIMARY="0.15c",
    COLOR_NAN="white",
)

fig.grdimage(
    grid=grid_to_plot,
    cmap=CPT_FILE,
    projection=PROJ,
    region=REGION,
    shading=shade,
)

# 海岸线 / 陆地
if SEA_ONLY:
    fig.coast(
        land="gray90",
        shorelines="0.25p,black",
        borders="1/0.15p,gray40",
        frame=["WSen", "xafg", "yafg"],
    )
else:
    fig.coast(
        shorelines="0.25p,black",
        borders="1/0.15p,gray40",
        frame=["WSen", "xafg", "yafg"],
    )

# ================= OJP 外轮廓 =================
if LINES_CSV and os.path.exists(LINES_CSV):
    dfL = pd.read_csv(LINES_CSV)
    required_cols = {"lon", "lat"}
    assert required_cols.issubset(dfL.columns), \
        f"LINES_CSV 需要包含列: {required_cols}"
    if "line_id" in dfL.columns:
        for lid, g in dfL.groupby("line_id"):
            fig.plot(x=g["lon"], y=g["lat"], pen="1.8p,black")
    else:
        fig.plot(x=dfL["lon"], y=dfL["lat"], pen="1.8p,black")

if LINES_SHP and os.path.exists(LINES_SHP):
    fig.plot(data=LINES_SHP, pen="1.8p,black")
# =====================================================

# ======= 叠加前人地震测线 =======
if MCS_LINES_CSV and os.path.exists(MCS_LINES_CSV):
    mcs = pd.read_csv(MCS_LINES_CSV)
    required_cols = {"lon", "lat", "line_name"}
    assert required_cols.issubset(mcs.columns), \
        f"MCS_LINES_CSV 需要包含列: {required_cols}, 实际为 {mcs.columns}"

    style_map = {

        "Kaiei_KR05-01": "2p,black",
        "KH98-01_a":     "2p,white",
        "KH98-01_b":     "2p,white",
        "KH98-01_c":     "2p,white",
        "KH98-01_d":     "2p,white",
        "EW95-11":       "2p,purple",

        "Miura_2004":    "2p,red",


        "Mosher_TW_A":   "2p,green,.-",
        "Mosher_TW_B":   "2p,green,.-",
        "Mosher_Joides": "2p,green,.-",

        "Lyra_Basin":    "2p,blue",
        "Nauru_Basin":   "2p,blue",
    }

    for name, g in mcs.groupby("line_name"):


        if name in ["KH98-01_a", "KH98-01_b", "KH98-01_c", "KH98-01_d"]:
            fig.plot(x=g["lon"], y=g["lat"], pen="2.8p,black")

        pen = style_map.get(name, "0.8p,gray50")
        fig.plot(x=g["lon"], y=g["lat"], pen=pen)
# =====================================================

# 站位符号
dsdp = sites[sites["program"] == "DSDP"]
odp  = sites[sites["program"] == "ODP"]
if not dsdp.empty:
    fig.plot(
        x=dsdp["lon"],
        y=dsdp["lat"],
        style="s0.25c",
        fill="white",
        pen="0.6p,black",
    )
if not odp.empty:
    fig.plot(
        x=odp["lon"],
        y=odp["lat"],
        style="s0.28c",
        fill="gold",
        pen="0.6p,black",
    )

# 站位编号（可选）
if DRAW_LABELS:
    def put_label(x, y, text):
        fig.text(
            x=float(x),
            y=float(y),
            text=str(text),
            justify="BC",
            offset=LABEL_OFFSET,
            font=LABEL_FONT,
            fill=LABEL_FILL,
            clearance=LABEL_BOX,
            pen=LABEL_PEN,
        )

    lab = sites[sites["site_num"].isin(HIGHLIGHT_DSDP.union(HIGHLIGHT_ODP))].copy()
    for _, r in lab.iterrows():
        put_label(r["lon"], r["lat"], int(r["site_num"]))

# 色标
fig.colorbar(
    cmap=CPT_FILE,
    position="JBC+w14c/0.45c+o0c/0.8c+h",
    frame="a1000f500+lBathymetry/topography (m)",
)

# 输出：PNG + PDF
fig.savefig(OUTBASE + ".png", dpi=300)
fig.savefig(OUTBASE + ".pdf")
print(f"Saved: {OUTBASE}.png / .pdf")

sites.sort_values(["program", "site_num"]).to_csv(
    r"E:\wjy\Gravity\out\pic\OJP_sites_filtered.csv",
    index=False,
)
print("Saved: OJP_sites_filtered.csv")


Saved: E:\wjy\Gravity\out\pic\backup\OJP_seismic_plot.png / .pdf
Saved: OJP_sites_filtered.csv
